In [2]:
import numpy as np
import cv2
import os
from ultralytics import YOLO
from statistics import mode

In [15]:
model = YOLO("runs/detect/train7/weights/last.pt")
shape_score_dict = {"circle":20,"square":15,"triangle":10,"cross":5}
images_dir = "data/images/train/"


In [9]:
cap = cv2.VideoCapture("gst-launch-1.0 rtspsrc location=rtsp://192.168.1.103:8554/unicast latency=0 buffer-mode=auto ! decodebin ! videoconvert ! autovideosink",cv2.CAP_GSTREAMER)

In [10]:
ret, frame = cap.read()

In [12]:
ret

False

In [13]:
while True:
    ret, frame = cap.read()
    if not ret:
        break
    cv2.imshow('frame',frame)
    if cv2.waitKey(2)&0xff == ord('d'):
        break
cv2.destroyAllWindows()

In [16]:
def calculate_score(results):
    score = 0
    shapes_conversion_dict = results.names
    for obj in map(int,results.cpu().boxes.cls.int()):
        obj = int(obj)
        shape = shapes_conversion_dict[obj]
        score += shape_score_dict[shape]
    return score


In [ ]:
rtsp = "rtsp://192.168.1.102:8554/unicast"
cap = cv2.VideoCapture(0)
scores = []
while True:
    ret, frame = cap.read()
    results = model(frame,conf=0.5,show=False)
    scores.append(calculate_score(results[0]))
    if len(scores) == 50:
        scores.pop(0)
    score = np.median(scores)
    img = results[0].plot(labels=False)
    cv2.putText(img, f"Score: {score}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.imshow('det',img)
    if cv2.waitKey(1) == ord('d'):
        break
cv2.destroyAllWindows()
cap.release()   

In [19]:
for img_name in os.listdir(images_dir):
    img_path = os.path.join(images_dir,img_name)
    img = cv2.imread(img_path)
    img = model(img,conf=0.5,show=False)[0].plot(labels=False)
    cv2.imshow('det',img)
    if cv2.waitKey(0)&0xff == ord('d'):
        break
cv2.destroyAllWindows()


0: 1280x1248 7 crosss, 9 circles, 10 triangles, 7 squares, 61.8ms
Speed: 13.9ms preprocess, 61.8ms inference, 1.1ms postprocess per image at shape (1, 3, 1280, 1248)

0: 768x1280 8 crosss, 10 circles, 9 triangles, 8 squares, 38.0ms
Speed: 5.2ms preprocess, 38.0ms inference, 1.5ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 (no detections), 38.3ms
Speed: 4.5ms preprocess, 38.3ms inference, 0.7ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 7 crosss, 9 circles, 10 triangles, 7 squares, 37.4ms
Speed: 4.7ms preprocess, 37.4ms inference, 1.1ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 8 crosss, 10 circles, 9 triangles, 8 squares, 37.4ms
Speed: 5.3ms preprocess, 37.4ms inference, 1.2ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 8 crosss, 10 circles, 9 triangles, 8 squares, 44.5ms
Speed: 4.8ms preprocess, 44.5ms inference, 1.3ms postprocess per image at shape (1, 3, 768, 1280)

0: 768x1280 7 crosss, 9 circles, 10 trian

In [13]:
def crop_images():
    for img_name in os.listdir(images_dir):
        img_path = os.path.join("shapes_pics",img_name)
        img = cv2.imread(img_path)
        orig = np.array(img)
        cv2.imwrite(os.path.join("out",img_name),img)
        # img = black_detection(img)
    #     cv2.imshow('orig',cv2.resize(orig,None,fx=0.5,fy=0.5))
    #     # cv2.imshow('img',cv2.resize(img,None,fx=0.5,fy=0.5))
    #     if cv2.waitKey(0)&0xff == ord('d'):
    #         break
    # cv2.destroyAllWindows()
# crop_images()

[ WARN:0@358.461] global loadsave.cpp:248 findDecoder imread_('shapes_pics/1.jpg'): can't open/read file: check file path/integrity


error: OpenCV(4.8.0) /io/opencv/modules/imgcodecs/src/loadsave.cpp:787: error: (-215:Assertion failed) !_img.empty() in function 'imwrite'


In [20]:
# don't use this function
def black_detection(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    lower_black = np.array([0, 0, 0], dtype=np.uint8)
    upper_black = np.array([255, 200, 255], dtype=np.uint8)
    black_mask = cv2.inRange(hsv, lower_black, upper_black)
    # black_detected = cv2.bitwise_and(img, img, mask=black_mask)
    return black_mask

In [22]:
# capture atef's mobile video stream
cap = cv2.VideoCapture("rtsp://:8080/unicast")
while True:
    ret, frame = cap.read()
    frame = black_detection(frame)
    cv2.imshow('det',frame)
    if cv2.waitKey(1) == ord('d'):
        break
cv2.destroyAllWindows()
cap.release()   

[tcp @ 0x55bfeb822b00] Connection to tcp://10.10.221.34:8080 failed: No route to host
[ERROR:0@1004.016] global cap.cpp:164 open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.8.0) /io/opencv/modules/videoio/src/cap_images.cpp:253: error: (-5:Bad argument) CAP_IMAGES: can't find starting number (in the name of file): https://10.10.221.34:8080/unicast in function 'icvExtractPattern'




error: OpenCV(4.8.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
